In [2]:
# Classification
import pandas as pd
import numpy as np
import FinanceDataReader as fdr
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import pandas_ta
import talib
import datetime
from matplotlib import rc
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import KFold, cross_val_score, train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier, StackingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.ensemble import RandomForestClassifier
# from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import optuna
from sklearn.ensemble import HistGradientBoostingClassifier
from pykrx import stock
from scipy import stats

### Kospi

In [3]:
ks = yf.download('^KS11', start='2000-01-01', end = '2023-02-01')
kq = yf.download('^KQ11', start='2000-01-01', end = '2023-02-01')
ks200 = yf.download('^KS200', start='2000-01-01', end = '2023-02-01')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
# 각 연도별로 데이터 가져오기
def sep_date(data):
    santa = []
    for year in range(2000, 2023):
        year_data = data[data.index.year == year]
        year_data_next = data[data.index.year == year + 1]

        # 연말 5일 전 계산
        end_of_year = year_data.tail(5).index[0]

        # 연초 2일 계산
        start_of_year = year_data_next.head(2).index[-1]

        # 연속된 데이터 추출
        continuous_data = data.loc[end_of_year:start_of_year]
        rest_data = data[:end_of_year]
        # 결과 출력 또는 원하는 작업 수행
        santa.append(continuous_data)
    return santa


In [6]:
def make_1d_pct(df):
    df['1d_pct'] = ((df['Close'] - df['Open']) / df['Open']) * 100
    return df
    

In [7]:
def mannwhitneyu_test(df):
    santa_period = sep_date(df)
    santa_period = pd.concat(santa_period)
    rest_period = df[~df.index.isin(santa_period.index)]
    santa_period = make_1d_pct(santa_period)
    rest_period = make_1d_pct(rest_period)
    print(santa_period[['1d_pct', 'Volume']].describe())
    print(rest_period[['1d_pct', 'Volume']].describe())
    U_statistic, p_value = stats.mannwhitneyu(santa_period['1d_pct'], rest_period['1d_pct'])
    print(p_value)

In [15]:
mannwhitneyu_test(ks)

           1d_pct        Volume
count  161.000000  1.610000e+02
mean    -0.001272  4.110348e+05
std      1.105579  2.109544e+05
min     -3.760335  1.477000e+05
25%     -0.594009  2.832000e+05
50%      0.067491  3.590000e+05
75%      0.501923  4.580000e+05
max      3.861033  1.519900e+06
            1d_pct        Volume
count  5531.000000  5.531000e+03
mean     -0.038983  4.759988e+05
std       1.148376  2.641589e+05
min     -10.729565  1.363000e+05
25%      -0.558011  3.153000e+05
50%      -0.023817  4.024000e+05
75%       0.521659  5.430000e+05
max       8.977477  3.455500e+06
0.8412440936361698


### Kospi 200 종목 불러오기

In [11]:
ks_stock_list = fdr.StockListing('KOSPI')
ks_code_list = ks_stock_list['Code'].apply(lambda x : str(x) + '.KS').tolist()

In [12]:
ks200_html = pd.read_html('https://ko.wikipedia.org/wiki/%EC%BD%94%EC%8A%A4%ED%94%BC_200')[-1]
ks200_html.columns = ['Sector', 'Name']
# print(ks200_html.columns)
ks200 = ks200_html.iloc[1:,]
ks200.columns = ['sector', 'stock']
ks200_stock_name = []
for i in range(1, 12):
    data = ks200['stock'][i].split()
    sector_name = ks200_html['Sector'][i]
    for j in data:
        dicts = {'Name' : j, 'sector' : sector_name}
        ks200_stock_name.append(dicts)
ks200 = pd.DataFrame(ks200_stock_name)

In [13]:
# 시장별 데이터 가져오기 : 'KOSPI', 'KODAQ', 'KONEX'
stocks_kospi = fdr.StockListing('KOSPI')[['Code', 'Name']]
# 앞 5개 뒷부분 5개 보여준다.
ks200 = pd.merge(ks200, stocks_kospi, on='Name')

In [14]:
ks200

,Name,sector,Code
0,DL이앤씨,건설,375500
1,GS건설,건설,006360
2,대우건설,건설,047040
3,삼성물산,건설,028260
4,삼성엔지니어링,건설,028050
...,...,...,...
194,유한양행,헬스케어,000100
195,종근당,헬스케어,185750
196,한미사이언스,헬스케어,008930
197,한미약품,헬스케어,128940


In [12]:
ks200['Code'] = ks200['Code'].apply(lambda x : str(x) + '.KS')

In [20]:
ks200['Sector'] = ks

,Name,Code,Sector
0,DL이앤씨,375500.KS,False
1,GS건설,006360.KS,False
2,대우건설,047040.KS,False
3,삼성물산,028260.KS,False
4,삼성엔지니어링,028050.KS,False
...,...,...,...
194,유한양행,000100.KS,False
195,종근당,185750.KS,False
196,한미사이언스,008930.KS,False
197,한미약품,128940.KS,False
